In [4]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Parameters
img_width, img_height = 224, 224
batch_size = 32
epochs = 50

# Directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data generator for training data
#Data Aug....
# Data generator for training data with validation split
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% validation split

# Separate generator for training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Specify 'training' subset

# Separate generator for validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Specify 'validation' subset



# Data generator for test data
test_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

# Create a DataFrame with the filenames
test_df = pd.DataFrame({"Filename": test_files})

#Data Aug
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.
Found 100 validated image filenames.


In [4]:
#####PRETRAINED MODEL

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters, stride=1, use_projection=False):
    shortcut = x
    if use_projection:
        shortcut = Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # First convolutional layer
    x = Conv2D(filters, kernel_size=(3, 3), strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Add shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def ResNet50(input_shape=(227, 227, 3), num_classes=5):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv2D(64, kernel_size=(7, 7), strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64, use_projection=True)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)

    x = residual_block(x, filters=128, stride=2, use_projection=True)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=128)

    x = residual_block(x, filters=256, stride=2, use_projection=True)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=256)

    x = residual_block(x, filters=512, stride=2, use_projection=True)
    x = residual_block(x, filters=512)
    x = residual_block(x, filters=512)

    # Final layers
    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create ResNet50 model
resnet50_model = ResNet50()

# Display the model summary
resnet50_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 114, 114, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 114, 114, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [5]:

resnet50_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])


In [6]:
               
resnet50_model.fit(train_generator, 
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=validation_generator, 
    validation_steps=len(validation_generator),
   ) 


Epoch 1/40
248/248 [==============================] - 95s 342ms/step - loss: 1.3395 - accuracy: 0.4865 - val_loss: 2.7333 - val_accuracy: 0.2061
Epoch 2/40
248/248 [==============================] - 80s 324ms/step - loss: 1.0810 - accuracy: 0.5756 - val_loss: 1.6324 - val_accuracy: 0.3833
Epoch 3/40
248/248 [==============================] - 77s 311ms/step - loss: 1.0123 - accuracy: 0.6034 - val_loss: 1.3618 - val_accuracy: 0.4611
Epoch 4/40
248/248 [==============================] - 80s 320ms/step - loss: 0.9654 - accuracy: 0.6221 - val_loss: 1.5169 - val_accuracy: 0.4025
Epoch 5/40
248/248 [==============================] - 79s 319ms/step - loss: 0.9078 - accuracy: 0.6484 - val_loss: 1.2966 - val_accuracy: 0.4652
Epoch 6/40
248/248 [==============================] - 78s 314ms/step - loss: 0.9135 - accuracy: 0.6441 - val_loss: 1.4587 - val_accuracy: 0.4505
Epoch 7/40
248/248 [==============================] - 71s 285ms/step - loss: 0.8455 - accuracy: 0.6734 - val_loss: 1.1951 - val_ac

####output
Epoch 1/40
248/248 [==============================] - 95s 342ms/step - loss: 1.3395 - accuracy: 0.4865 - val_loss: 2.7333 - val_accuracy: 0.2061
Epoch 2/40
248/248 [==============================] - 80s 324ms/step - loss: 1.0810 - accuracy: 0.5756 - val_loss: 1.6324 - val_accuracy: 0.3833
Epoch 3/40
248/248 [==============================] - 77s 311ms/step - loss: 1.0123 - accuracy: 0.6034 - val_loss: 1.3618 - val_accuracy: 0.4611
Epoch 4/40
248/248 [==============================] - 80s 320ms/step - loss: 0.9654 - accuracy: 0.6221 - val_loss: 1.5169 - val_accuracy: 0.4025
Epoch 5/40
248/248 [==============================] - 79s 319ms/step - loss: 0.9078 - accuracy: 0.6484 - val_loss: 1.2966 - val_accuracy: 0.4652
Epoch 6/40
248/248 [==============================] - 78s 314ms/step - loss: 0.9135 - accuracy: 0.6441 - val_loss: 1.4587 - val_accuracy: 0.4505
Epoch 7/40
248/248 [==============================] - 71s 285ms/step - loss: 0.8455 - accuracy: 0.6734 - val_loss: 1.1951 - val_accuracy: 0.5333
Epoch 8/40
248/248 [==============================] - 81s 326ms/step - loss: 0.8332 - accuracy: 0.6843 - val_loss: 1.1367 - val_accuracy: 0.5591
Epoch 9/40
248/248 [==============================] - 73s 295ms/step - loss: 0.8010 - accuracy: 0.6994 - val_loss: 1.2691 - val_accuracy: 0.5364
Epoch 10/40
248/248 [==============================] - 73s 295ms/step - loss: 0.7842 - accuracy: 0.7081 - val_loss: 1.2890 - val_accuracy: 0.5045
Epoch 11/40
248/248 [==============================] - 71s 287ms/step - loss: 0.8036 - accuracy: 0.6934 - val_loss: 0.9537 - val_accuracy: 0.6571
Epoch 12/40
248/248 [==============================] - 74s 298ms/step - loss: 0.7489 - accuracy: 0.7178 - val_loss: 0.9610 - val_accuracy: 0.6222
Epoch 13/40
248/248 [==============================] - 71s 284ms/step - loss: 0.7406 - accuracy: 0.7232 - val_loss: 1.8443 - val_accuracy: 0.4990
Epoch 14/40
248/248 [==============================] - 73s 293ms/step - loss: 0.7087 - accuracy: 0.7348 - val_loss: 1.1407 - val_accuracy: 0.6035
Epoch 15/40
248/248 [==============================] - 72s 291ms/step - loss: 0.6940 - accuracy: 0.7399 - val_loss: 1.6511 - val_accuracy: 0.4298
Epoch 16/40
248/248 [==============================] - 74s 299ms/step - loss: 0.6822 - accuracy: 0.7423 - val_loss: 0.9037 - val_accuracy: 0.6591
Epoch 17/40
248/248 [==============================] - 71s 285ms/step - loss: 0.6816 - accuracy: 0.7446 - val_loss: 1.8166 - val_accuracy: 0.4909
Epoch 18/40
248/248 [==============================] - 74s 296ms/step - loss: 0.6692 - accuracy: 0.7491 - val_loss: 0.8350 - val_accuracy: 0.7015
Epoch 19/40
248/248 [==============================] - 71s 287ms/step - loss: 0.6396 - accuracy: 0.7605 - val_loss: 1.0367 - val_accuracy: 0.6051
Epoch 20/40
248/248 [==============================] - 77s 309ms/step - loss: 0.6412 - accuracy: 0.7611 - val_loss: 1.1777 - val_accuracy: 0.6263
Epoch 21/40
248/248 [==============================] - 72s 288ms/step - loss: 0.6482 - accuracy: 0.7598 - val_loss: 10.3072 - val_accuracy: 0.5268
Epoch 22/40
248/248 [==============================] - 74s 298ms/step - loss: 0.6120 - accuracy: 0.7679 - val_loss: 1.8083 - val_accuracy: 0.6247
Epoch 23/40
248/248 [==============================] - 72s 291ms/step - loss: 0.6006 - accuracy: 0.7754 - val_loss: 1.1416 - val_accuracy: 0.6237
Epoch 24/40
248/248 [==============================] - 79s 317ms/step - loss: 0.5943 - accuracy: 0.7785 - val_loss: 2.1000 - val_accuracy: 0.4323
Epoch 25/40
248/248 [==============================] - 71s 284ms/step - loss: 0.5797 - accuracy: 0.7860 - val_loss: 0.9031 - val_accuracy: 0.6727
Epoch 26/40
248/248 [==============================] - 73s 295ms/step - loss: 0.5711 - accuracy: 0.7843 - val_loss: 1.1866 - val_accuracy: 0.5803
Epoch 27/40
248/248 [==============================] - 72s 291ms/step - loss: 0.5482 - accuracy: 0.7994 - val_loss: 1.1587 - val_accuracy: 0.6182
Epoch 28/40
248/248 [==============================] - 74s 296ms/step - loss: 0.5567 - accuracy: 0.8001 - val_loss: 1.0715 - val_accuracy: 0.6141
Epoch 29/40
248/248 [==============================] - 71s 286ms/step - loss: 0.5336 - accuracy: 0.8033 - val_loss: 0.9237 - val_accuracy: 0.6515
Epoch 30/40
248/248 [==============================] - 74s 298ms/step - loss: 0.5157 - accuracy: 0.8128 - val_loss: 0.7923 - val_accuracy: 0.7187
Epoch 31/40
248/248 [==============================] - 78s 312ms/step - loss: 0.5095 - accuracy: 0.8135 - val_loss: 1.2877 - val_accuracy: 0.5566
Epoch 32/40
248/248 [==============================] - 78s 314ms/step - loss: 0.4935 - accuracy: 0.8203 - val_loss: 1.0260 - val_accuracy: 0.6404
Epoch 33/40
248/248 [==============================] - 70s 283ms/step - loss: 0.4944 - accuracy: 0.8187 - val_loss: 1.0167 - val_accuracy: 0.6485
Epoch 34/40
248/248 [==============================] - 72s 288ms/step - loss: 0.4812 - accuracy: 0.8277 - val_loss: 1.0302 - val_accuracy: 0.6399
Epoch 35/40
248/248 [==============================] - 71s 287ms/step - loss: 0.4611 - accuracy: 0.8311 - val_loss: 0.9923 - val_accuracy: 0.6621
Epoch 36/40
248/248 [==============================] - 73s 292ms/step - loss: 0.4513 - accuracy: 0.8298 - val_loss: 1.2575 - val_accuracy: 0.6035
Epoch 37/40
248/248 [==============================] - 69s 279ms/step - loss: 0.4471 - accuracy: 0.8369 - val_loss: 0.9494 - val_accuracy: 0.6682
Epoch 38/40
248/248 [==============================] - 77s 309ms/step - loss: 0.4359 - accuracy: 0.8405 - val_loss: 0.8327 - val_accuracy: 0.7545
Epoch 39/40
248/248 [==============================] - 70s 282ms/step - loss: 0.4237 - accuracy: 0.8489 - val_loss: 1.1590 - val_accuracy: 0.6389
Epoch 40/40
248/248 [==============================] - 73s 294ms/step - loss: 0.4163 - accuracy: 0.8485 - val_loss: 0.8049 - val_accuracy: 0.7152


In [ ]:
from tensorflow.keras import layers, Model, Input

def residual_block(x, filters, stride=1, use_projection=False):
    shortcut = x
    if use_projection:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    # First convolutional layer
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second convolutional layer
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add dropout for regularization
    x = layers.Dropout(0.3)(x)

    # Add shortcut to the output
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def ResNet50_modified(input_shape=(224, 224, 3), num_classes=5):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

    # Reduced number of residual blocks for brevity
    x = residual_block(x, filters=64, use_projection=True)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, stride=2, use_projection=True)
    x = residual_block(x, filters=128)

    # Fewer residual blocks for illustration purposes
    x = residual_block(x, filters=256, stride=2, use_projection=True)
    x = residual_block(x, filters=256)

    x = layers.AveragePooling2D(pool_size=(7, 7))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the modified ResNet50 model
modified_resnet50_model = ResNet50_modified()

# Display the model summary
modified_resnet50_model.summary()


In [ ]:

modified_resnet50_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])
               
modified_resnet50_model.fit(train_generator, 
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator, 
    validation_steps=len(validation_generator),
   ) 


In [8]:
accuracy = resnet50_model.evaluate(test_generator)
# Make predictions on test data
predictions = resnet50_model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Get only the filenames without the directory path
test_generator.filenames = [os.path.basename(file) for file in test_generator.filenames]

# Add 1 to each predicted class to shift the numbering
predicted_classes_adjusted = predicted_classes + 1
# Create a DataFrame to store filenames without paths and predicted classes
results = pd.DataFrame({"image_id": test_generator.filenames, "label": predicted_classes_adjusted})

# Save results to a CSV file
results.to_csv('submission_resnet_GpU.csv', index=False)

# Display predicted classes
print(predicted_classes)

# Save the trained model
resnet50_model.save('trained_resnst50_model_GPU.h5')

4/4 [==============================] - 1s 162ms/step
[2 2 4 0 0 0 4 2 1 3 2 0 1 4 4 0 1 4 3 3 2 3 4 0 4 0 2 4 4 0 0 2 3 0 4 0 0
 1 0 2 0 2 3 2 4 3 2 1 2 0 4 1 1 1 1 1 1 1 4 1 1 0 1 1 3 3 1 2 1 4 3 1 1 1
 1 1 0 1 1 0 3 2 3 4 3 2 3 4 4 4 3 2 4 0 1 4 3 3 2 1]


In [6]:


# First CNN model
model1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])

# Second CNN model
model2 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(5, activation='softmax')
])

# Display the model summary
model1.summary()
# Display the model summary
model2.summary()
# Compile both models
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train both models
history1 = model1.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator)

history2 = model2.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator)

# Save the trained model
model1.save('saved_model1.h5')
model2.save('saved_model2.h5')



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 225, 225, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 55, 55, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 193600)            0         
                                                                 
 dense (Dense)               (None, 128)               2

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d (Conv2D)             (None, 225, 225, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 55, 55, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 193600)            0         
                                                                 
 dense (Dense)               (None, 128)               24780928  
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
=================================================================
Total params: 24,800,965
Trainable params: 24,800,965
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d_2 (Conv2D)           (None, 225, 225, 64)      1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 110, 110, 128)     73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 55, 55, 128)      0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 387200)            0         
                                                                 
 dense_2 (Dense)             (None, 256)               99123456  
                                                                 
 dense_3 (Dense)             (None, 5)                 1285      
                                                                 
=================================================================
Total params: 99,200,389
Trainable params: 99,200,389
Non-trainable params: 0


In [ ]:

# Load the trained model
#model1 = load_model('saved_model2.h5')

# Evaluate the model on the test data
test_loss, test_accuracy = model1.evaluate(test_generator)
)
predictions = model1.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Get only the filenames without the directory path
test_generator.filenames = [os.path.basename(file) for file in test_generator.filenames]

# Add 1 to each predicted class to shift the numbering
predicted_classes_adjusted = predicted_classes + 1
# Create a DataFrame to store filenames without paths and predicted classes
results = pd.DataFrame({"image_id": test_generator.filenames, "label": predicted_classes_adjusted})

# Save results to a CSV file
results.to_csv('submission_model1_GPU.csv', index=False)

In [ ]:

# Load the trained model
#model1 = load_model('saved_model2.h5')

# Evaluate the model on the test data
test_loss, test_accuracy = model2.evaluate(test_generator)

predictions = model2.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Get only the filenames without the directory path
test_generator.filenames = [os.path.basename(file) for file in test_generator.filenames]

# Add 1 to each predicted class to shift the numbering
predicted_classes_adjusted = predicted_classes + 1
# Create a DataFrame to store filenames without paths and predicted classes
results = pd.DataFrame({"image_id": test_generator.filenames, "label": predicted_classes_adjusted})

# Save results to a CSV file
results.to_csv('submission_model2_GPU.csv', index=False)

##############OUTPUT CNN 1
Epoch 1/50
248/248 [==============================] - 82s 310ms/step - loss: 1.5053 - accuracy: 0.4640 - val_loss: 1.2139 - val_accuracy: 0.5000
Epoch 2/50
248/248 [==============================] - 67s 272ms/step - loss: 1.0787 - accuracy: 0.5525 - val_loss: 1.1716 - val_accuracy: 0.5071
Epoch 3/50
248/248 [==============================] - 70s 280ms/step - loss: 1.0288 - accuracy: 0.5822 - val_loss: 1.1248 - val_accuracy: 0.5333
Epoch 4/50
248/248 [==============================] - 73s 293ms/step - loss: 0.9605 - accuracy: 0.6112 - val_loss: 1.0640 - val_accuracy: 0.5732
Epoch 5/50
248/248 [==============================] - 70s 281ms/step - loss: 0.9416 - accuracy: 0.6246 - val_loss: 1.0836 - val_accuracy: 0.5611
Epoch 6/50
248/248 [==============================] - 67s 270ms/step - loss: 0.9022 - accuracy: 0.6479 - val_loss: 1.0445 - val_accuracy: 0.5768
Epoch 7/50
248/248 [==============================] - 67s 270ms/step - loss: 0.8698 - accuracy: 0.6571 - val_loss: 1.1210 - val_accuracy: 0.5545
Epoch 8/50
248/248 [==============================] - 70s 279ms/step - loss: 0.8512 - accuracy: 0.6617 - val_loss: 1.0071 - val_accuracy: 0.5975
Epoch 9/50
248/248 [==============================] - 74s 300ms/step - loss: 0.8298 - accuracy: 0.6723 - val_loss: 1.0199 - val_accuracy: 0.6000
Epoch 10/50
248/248 [==============================] - 68s 274ms/step - loss: 0.8207 - accuracy: 0.6811 - val_loss: 1.0269 - val_accuracy: 0.6061
Epoch 11/50
248/248 [==============================] - 67s 269ms/step - loss: 0.7844 - accuracy: 0.6965 - val_loss: 1.0444 - val_accuracy: 0.5864
Epoch 12/50
248/248 [==============================] - 68s 273ms/step - loss: 0.7872 - accuracy: 0.6890 - val_loss: 1.0339 - val_accuracy: 0.6192
Epoch 13/50
248/248 [==============================] - 67s 271ms/step - loss: 0.7535 - accuracy: 0.7081 - val_loss: 0.9942 - val_accuracy: 0.6227
Epoch 14/50
248/248 [==============================] - 68s 272ms/step - loss: 0.7464 - accuracy: 0.7052 - val_loss: 1.0942 - val_accuracy: 0.5934
Epoch 15/50
248/248 [==============================] - 68s 272ms/step - loss: 0.7202 - accuracy: 0.7177 - val_loss: 1.0109 - val_accuracy: 0.6333
Epoch 16/50
248/248 [==============================] - 67s 271ms/step - loss: 0.7007 - accuracy: 0.7290 - val_loss: 0.9910 - val_accuracy: 0.6268
Epoch 17/50
248/248 [==============================] - 67s 271ms/step - loss: 0.6705 - accuracy: 0.7409 - val_loss: 1.0192 - val_accuracy: 0.6121
Epoch 18/50
248/248 [==============================] - 66s 267ms/step - loss: 0.6872 - accuracy: 0.7331 - val_loss: 1.0192 - val_accuracy: 0.6111
Epoch 19/50
248/248 [==============================] - 66s 267ms/step - loss: 0.6725 - accuracy: 0.7408 - val_loss: 1.0500 - val_accuracy: 0.6020
Epoch 20/50
248/248 [==============================] - 67s 268ms/step - loss: 0.6515 - accuracy: 0.7492 - val_loss: 1.0449 - val_accuracy: 0.5924
Epoch 21/50
248/248 [==============================] - 67s 267ms/step - loss: 0.6268 - accuracy: 0.7591 - val_loss: 1.0434 - val_accuracy: 0.6187
Epoch 22/50
248/248 [==============================] - 66s 267ms/step - loss: 0.6253 - accuracy: 0.7686 - val_loss: 1.0483 - val_accuracy: 0.6081
Epoch 23/50
248/248 [==============================] - 66s 266ms/step - loss: 0.6128 - accuracy: 0.7663 - val_loss: 1.0252 - val_accuracy: 0.6263
Epoch 24/50
248/248 [==============================] - 66s 266ms/step - loss: 0.5939 - accuracy: 0.7713 - val_loss: 1.0439 - val_accuracy: 0.6212
Epoch 25/50
248/248 [==============================] - 66s 267ms/step - loss: 0.5973 - accuracy: 0.7720 - val_loss: 0.9948 - val_accuracy: 0.6455
Epoch 26/50
248/248 [==============================] - 67s 267ms/step - loss: 0.5647 - accuracy: 0.7808 - val_loss: 1.0957 - val_accuracy: 0.6318
Epoch 27/50
248/248 [==============================] - 67s 270ms/step - loss: 0.5578 - accuracy: 0.7896 - val_loss: 1.0578 - val_accuracy: 0.6293
Epoch 28/50
248/248 [==============================] - 67s 271ms/step - loss: 0.5608 - accuracy: 0.7898 - val_loss: 1.0718 - val_accuracy: 0.6268
Epoch 29/50
248/248 [==============================] - 67s 269ms/step - loss: 0.5502 - accuracy: 0.7857 - val_loss: 1.0757 - val_accuracy: 0.6237
Epoch 30/50
248/248 [==============================] - 68s 273ms/step - loss: 0.5324 - accuracy: 0.7977 - val_loss: 1.0389 - val_accuracy: 0.6384
Epoch 31/50
248/248 [==============================] - 67s 268ms/step - loss: 0.5232 - accuracy: 0.8037 - val_loss: 1.0900 - val_accuracy: 0.6167
Epoch 32/50
248/248 [==============================] - 67s 271ms/step - loss: 0.5128 - accuracy: 0.8027 - val_loss: 1.0596 - val_accuracy: 0.6384
Epoch 33/50
248/248 [==============================] - 67s 268ms/step - loss: 0.4953 - accuracy: 0.8124 - val_loss: 1.1941 - val_accuracy: 0.6217
Epoch 34/50
248/248 [==============================] - 68s 271ms/step - loss: 0.4963 - accuracy: 0.8135 - val_loss: 1.1882 - val_accuracy: 0.6237
Epoch 35/50
248/248 [==============================] - 67s 270ms/step - loss: 0.4691 - accuracy: 0.8234 - val_loss: 1.1160 - val_accuracy: 0.6278
Epoch 36/50
248/248 [==============================] - 67s 270ms/step - loss: 0.4649 - accuracy: 0.8269 - val_loss: 1.1720 - val_accuracy: 0.6333
Epoch 37/50
248/248 [==============================] - 67s 271ms/step - loss: 0.4610 - accuracy: 0.8211 - val_loss: 1.1585 - val_accuracy: 0.6167
Epoch 38/50
248/248 [==============================] - 68s 272ms/step - loss: 0.4473 - accuracy: 0.8284 - val_loss: 1.2047 - val_accuracy: 0.6399
Epoch 39/50
248/248 [==============================] - 67s 271ms/step - loss: 0.4531 - accuracy: 0.8308 - val_loss: 1.2725 - val_accuracy: 0.5980
Epoch 40/50
248/248 [==============================] - 68s 272ms/step - loss: 0.4384 - accuracy: 0.8376 - val_loss: 1.1504 - val_accuracy: 0.6237
Epoch 41/50
248/248 [==============================] - 68s 272ms/step - loss: 0.4226 - accuracy: 0.8439 - val_loss: 1.2110 - val_accuracy: 0.6409
Epoch 42/50
248/248 [==============================] - 68s 272ms/step - loss: 0.4305 - accuracy: 0.8419 - val_loss: 1.1685 - val_accuracy: 0.6308
Epoch 43/50
248/248 [==============================] - 66s 267ms/step - loss: 0.4440 - accuracy: 0.8371 - val_loss: 1.1959 - val_accuracy: 0.6338
Epoch 44/50
248/248 [==============================] - 67s 271ms/step - loss: 0.4241 - accuracy: 0.8378 - val_loss: 1.1794 - val_accuracy: 0.6490
Epoch 45/50
248/248 [==============================] - 67s 267ms/step - loss: 0.3967 - accuracy: 0.8523 - val_loss: 1.1958 - val_accuracy: 0.6449
Epoch 46/50
248/248 [==============================] - 67s 270ms/step - loss: 0.3892 - accuracy: 0.8527 - val_loss: 1.2152 - val_accuracy: 0.6268
Epoch 47/50
248/248 [==============================] - 66s 267ms/step - loss: 0.3990 - accuracy: 0.8520 - val_loss: 1.2408 - val_accuracy: 0.6449
Epoch 48/50
248/248 [==============================] - 67s 270ms/step - loss: 0.3922 - accuracy: 0.8568 - val_loss: 1.2542 - val_accuracy: 0.6222
Epoch 49/50
248/248 [==============================] - 68s 273ms/step - loss: 0.3802 - accuracy: 0.8600 - val_loss: 1.2391 - val_accuracy: 0.6399
Epoch 50/50
248/248 [==============================] - 69s 276ms/step - loss: 0.3767 - accuracy: 0.8602 - val_loss: 1.2961 - val_accuracy: 0.6460


##############OUTPUT CNN 2

Epoch 1/50
248/248 [==============================] - 73s 285ms/step - loss: 1.6925 - accuracy: 0.4148 - val_loss: 1.2959 - val_accuracy: 0.4571
Epoch 2/50
248/248 [==============================] - 69s 279ms/step - loss: 1.1123 - accuracy: 0.5295 - val_loss: 1.2232 - val_accuracy: 0.4874
Epoch 3/50
248/248 [==============================] - 69s 278ms/step - loss: 1.0485 - accuracy: 0.5648 - val_loss: 1.1747 - val_accuracy: 0.5076
Epoch 4/50
248/248 [==============================] - 68s 274ms/step - loss: 1.0103 - accuracy: 0.5835 - val_loss: 1.1392 - val_accuracy: 0.5293
Epoch 5/50
248/248 [==============================] - 68s 274ms/step - loss: 0.9832 - accuracy: 0.5982 - val_loss: 1.1340 - val_accuracy: 0.5253
Epoch 6/50
248/248 [==============================] - 68s 273ms/step - loss: 0.9510 - accuracy: 0.6122 - val_loss: 1.0915 - val_accuracy: 0.5399
Epoch 7/50
248/248 [==============================] - 71s 284ms/step - loss: 0.9391 - accuracy: 0.6169 - val_loss: 1.0892 - val_accuracy: 0.5414
Epoch 8/50
248/248 [==============================] - 69s 277ms/step - loss: 0.9263 - accuracy: 0.6256 - val_loss: 1.0596 - val_accuracy: 0.5636
Epoch 9/50
248/248 [==============================] - 69s 278ms/step - loss: 0.9160 - accuracy: 0.6304 - val_loss: 1.0767 - val_accuracy: 0.5384
Epoch 10/50
248/248 [==============================] - 68s 273ms/step - loss: 0.8853 - accuracy: 0.6391 - val_loss: 1.0828 - val_accuracy: 0.5955
Epoch 11/50
248/248 [==============================] - 68s 273ms/step - loss: 0.8754 - accuracy: 0.6538 - val_loss: 1.0366 - val_accuracy: 0.5783
Epoch 12/50
248/248 [==============================] - 68s 273ms/step - loss: 0.8455 - accuracy: 0.6668 - val_loss: 1.0969 - val_accuracy: 0.5525
Epoch 13/50
248/248 [==============================] - 68s 273ms/step - loss: 0.8403 - accuracy: 0.6711 - val_loss: 1.0317 - val_accuracy: 0.6081
Epoch 14/50
248/248 [==============================] - 68s 274ms/step - loss: 0.8215 - accuracy: 0.6771 - val_loss: 1.0553 - val_accuracy: 0.5712
Epoch 15/50
248/248 [==============================] - 68s 273ms/step - loss: 0.8075 - accuracy: 0.6852 - val_loss: 1.0341 - val_accuracy: 0.5783
Epoch 16/50
248/248 [==============================] - 68s 273ms/step - loss: 0.7861 - accuracy: 0.6982 - val_loss: 1.0204 - val_accuracy: 0.6010
Epoch 17/50
248/248 [==============================] - 67s 270ms/step - loss: 0.7778 - accuracy: 0.6977 - val_loss: 1.0427 - val_accuracy: 0.5955
Epoch 18/50
248/248 [==============================] - 67s 271ms/step - loss: 0.7753 - accuracy: 0.6999 - val_loss: 1.0495 - val_accuracy: 0.6040
Epoch 19/50
248/248 [==============================] - 68s 272ms/step - loss: 0.7502 - accuracy: 0.7076 - val_loss: 1.0821 - val_accuracy: 0.5869
Epoch 20/50
248/248 [==============================] - 68s 272ms/step - loss: 0.7332 - accuracy: 0.7121 - val_loss: 1.0002 - val_accuracy: 0.6010
Epoch 21/50
248/248 [==============================] - 68s 272ms/step - loss: 0.7245 - accuracy: 0.7189 - val_loss: 1.0413 - val_accuracy: 0.5934
Epoch 22/50
248/248 [==============================] - 69s 276ms/step - loss: 0.7015 - accuracy: 0.7316 - val_loss: 0.9862 - val_accuracy: 0.6263
Epoch 23/50
248/248 [==============================] - 68s 274ms/step - loss: 0.7037 - accuracy: 0.7311 - val_loss: 1.0053 - val_accuracy: 0.6318
Epoch 24/50
248/248 [==============================] - 68s 273ms/step - loss: 0.6680 - accuracy: 0.7489 - val_loss: 1.0489 - val_accuracy: 0.6333
Epoch 25/50
248/248 [==============================] - 68s 272ms/step - loss: 0.6520 - accuracy: 0.7489 - val_loss: 1.0403 - val_accuracy: 0.6308
Epoch 26/50
248/248 [==============================] - 68s 272ms/step - loss: 0.6374 - accuracy: 0.7529 - val_loss: 1.0282 - val_accuracy: 0.6268
Epoch 27/50
248/248 [==============================] - 67s 271ms/step - loss: 0.6251 - accuracy: 0.7585 - val_loss: 0.9829 - val_accuracy: 0.6460
Epoch 28/50
248/248 [==============================] - 67s 271ms/step - loss: 0.6132 - accuracy: 0.7663 - val_loss: 1.0164 - val_accuracy: 0.6338
Epoch 29/50
248/248 [==============================] - 69s 276ms/step - loss: 0.6012 - accuracy: 0.7649 - val_loss: 0.9949 - val_accuracy: 0.6444
Epoch 30/50
248/248 [==============================] - 68s 273ms/step - loss: 0.6032 - accuracy: 0.7713 - val_loss: 1.0670 - val_accuracy: 0.6217
Epoch 31/50
248/248 [==============================] - 68s 273ms/step - loss: 0.5896 - accuracy: 0.7766 - val_loss: 1.1077 - val_accuracy: 0.6162
Epoch 32/50
248/248 [==============================] - 68s 274ms/step - loss: 0.5650 - accuracy: 0.7857 - val_loss: 1.0547 - val_accuracy: 0.6343
Epoch 33/50
248/248 [==============================] - 68s 274ms/step - loss: 0.5666 - accuracy: 0.7822 - val_loss: 1.0428 - val_accuracy: 0.6348
Epoch 34/50
248/248 [==============================] - 68s 272ms/step - loss: 0.5684 - accuracy: 0.7809 - val_loss: 1.0620 - val_accuracy: 0.6126
Epoch 35/50
248/248 [==============================] - 68s 272ms/step - loss: 0.5549 - accuracy: 0.7905 - val_loss: 1.0403 - val_accuracy: 0.6520
Epoch 36/50
248/248 [==============================] - 68s 272ms/step - loss: 0.5328 - accuracy: 0.7938 - val_loss: 1.0144 - val_accuracy: 0.6404
Epoch 37/50
248/248 [==============================] - 69s 277ms/step - loss: 0.5157 - accuracy: 0.8093 - val_loss: 1.1232 - val_accuracy: 0.6313
Epoch 38/50
248/248 [==============================] - 68s 273ms/step - loss: 0.5139 - accuracy: 0.8057 - val_loss: 1.1110 - val_accuracy: 0.6313
Epoch 39/50
248/248 [==============================] - 71s 284ms/step - loss: 0.5040 - accuracy: 0.8104 - val_loss: 1.1681 - val_accuracy: 0.6313
Epoch 40/50
248/248 [==============================] - 70s 280ms/step - loss: 0.4912 - accuracy: 0.8136 - val_loss: 1.1156 - val_accuracy: 0.6323
Epoch 41/50
248/248 [==============================] - 77s 308ms/step - loss: 0.4811 - accuracy: 0.8210 - val_loss: 1.0455 - val_accuracy: 0.6535
Epoch 42/50
248/248 [==============================] - 75s 303ms/step - loss: 0.4659 - accuracy: 0.8275 - val_loss: 1.0881 - val_accuracy: 0.6429
Epoch 43/50
248/248 [==============================] - 69s 279ms/step - loss: 0.4636 - accuracy: 0.8301 - val_loss: 1.1448 - val_accuracy: 0.6253
Epoch 44/50
248/248 [==============================] - 73s 294ms/step - loss: 0.4518 - accuracy: 0.8314 - val_loss: 1.2463 - val_accuracy: 0.6247
Epoch 45/50
248/248 [==============================] - 69s 276ms/step - loss: 0.4508 - accuracy: 0.8333 - val_loss: 1.2164 - val_accuracy: 0.6389
Epoch 46/50
248/248 [==============================] - 68s 272ms/step - loss: 0.4408 - accuracy: 0.8332 - val_loss: 1.2025 - val_accuracy: 0.6268
Epoch 47/50
248/248 [==============================] - 68s 274ms/step - loss: 0.4423 - accuracy: 0.8351 - val_loss: 1.2170 - val_accuracy: 0.6253
Epoch 48/50
248/248 [==============================] - 70s 281ms/step - loss: 0.4202 - accuracy: 0.8466 - val_loss: 1.1815 - val_accuracy: 0.6116
Epoch 49/50
248/248 [==============================] - 69s 276ms/step - loss: 0.4171 - accuracy: 0.8489 - val_loss: 1.1858 - val_accuracy: 0.6374
Epoch 50/50
248/248 [==============================] - 68s 272ms/step - loss: 0.3937 - accuracy: 0.8578 - val_loss: 1.3639 - val_accuracy: 0.6263


In [ ]:
############TRANFORMER

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import StratifiedKFold



import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

c:\Users\badr0\.conda\envs\py310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\badr0\.conda\envs\py310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want

In [4]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        layers.experimental.preprocessing.Resizing(224, 224),
#         layers.experimental.preprocessing.RandomFlip("horizontal"),
#         layers.experimental.preprocessing.RandomRotation(factor=0.02),
#         layers.experimental.preprocessing.RandomZoom(
#             height_factor=0.2, width_factor=0.2
#         ),
    ],
    name="data_augmentation",
)

In [5]:
from vit_keras import vit, utils

def create_vit_classifier():
    inputs = layers.Input(shape=(224,224,3))
    # Augment data.
    augmented = data_augmentation(inputs)


    pre_trained_model = vit.vit_l32(
        image_size=224,
        activation='relu',
        pretrained=True,
        include_top=True,
        pretrained_top=True,
        classes=5
    )
    pre_trained_model.trainable = False
    features = pre_trained_model(augmented)
    features = layers.Dense(512, activation=layers.PReLU())(features)
    features = layers.Dropout(0.2)(features)
    features = layers.Dense(256, activation=layers.PReLU())(features)
    features = layers.Dropout(0.2)(features)
    features = layers.Dense(128, activation=layers.PReLU())(features)
    features = layers.Dropout(0.2)(features)

    outputs = layers.Dense(5, activation="softmax")(features)
    model = keras.Model(inputs=inputs,outputs=outputs)
    return model

In [6]:
def run_experiment(model):



    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




    history = model.fit(
        train_generator,
        batch_size=batch_size,
        epochs=10,
        validation_data=validation_generator,

    )
    

    model.trainable = True
    history = model.fit(
        train_generator,
        batch_size=batch_size,
        epochs=10 // 2,

        validation_data=validation_generator,

    )

    accuracy = model.evaluate(test_generator)
    


    #model.save("./vit_best_model.h5")
    return history

vit_classifier = create_vit_classifier()
vit_classifier.summary()
history = run_experiment(vit_classifier)

c:\Users\badr0\.conda\envs\py310\lib\site-packages\vit_keras\vit.py:139: UserWarning: Can only use pretrained_top with imagenet21k+imagenet2012 if classes = 1000. Setting manually.
  warnings.warn(
c:\Users\badr0\.conda\envs\py310\lib\site-packages\vit_keras\utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 7, 7
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      7         
 al)                                                             
                                                                 
 vit-l32 (Functional)        (None, 1000)              306535400 
                                                                 
 dense (Dense)               (None, 512)               513024    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131584    
                                                             

TRANSFORMER
Epoch 1/40
248/248 [==============================] - 104s 338ms/step - loss: 0.8277 - accuracy: 0.6889 - val_loss: 0.5984 - val_accuracy: 0.7874
Epoch 2/40
248/248 [==============================] - 77s 308ms/step - loss: 0.5821 - accuracy: 0.7907 - val_loss: 0.5760 - val_accuracy: 0.7960
Epoch 3/40
248/248 [==============================] - 77s 310ms/step - loss: 0.5331 - accuracy: 0.8080 - val_loss: 0.5695 - val_accuracy: 0.7869
Epoch 4/40
248/248 [==============================] - 77s 309ms/step - loss: 0.4952 - accuracy: 0.8231 - val_loss: 0.5507 - val_accuracy: 0.8066
Epoch 5/40
248/248 [==============================] - 77s 309ms/step - loss: 0.4642 - accuracy: 0.8309 - val_loss: 0.4866 - val_accuracy: 0.8293
Epoch 6/40
248/248 [==============================] - 76s 308ms/step - loss: 0.4453 - accuracy: 0.8402 - val_loss: 0.5154 - val_accuracy: 0.8152
Epoch 7/40
248/248 [==============================] - 77s 308ms/step - loss: 0.4361 - accuracy: 0.8441 - val_loss: 0.5189 - val_accuracy: 0.8192
Epoch 8/40
248/248 [==============================] - 77s 311ms/step - loss: 0.4208 - accuracy: 0.8494 - val_loss: 0.4994 - val_accuracy: 0.8187
Epoch 9/40
248/248 [==============================] - 77s 310ms/step - loss: 0.4105 - accuracy: 0.8556 - val_loss: 0.5434 - val_accuracy: 0.8172
Epoch 10/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3963 - accuracy: 0.8591 - val_loss: 0.4925 - val_accuracy: 0.8222
Epoch 11/40
248/248 [==============================] - 77s 310ms/step - loss: 0.3941 - accuracy: 0.8629 - val_loss: 0.4630 - val_accuracy: 0.8288
Epoch 12/40
248/248 [==============================] - 76s 308ms/step - loss: 0.3822 - accuracy: 0.8662 - val_loss: 0.4490 - val_accuracy: 0.8419
Epoch 13/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3866 - accuracy: 0.8586 - val_loss: 0.4578 - val_accuracy: 0.8404
Epoch 14/40
248/248 [==============================] - 76s 305ms/step - loss: 0.3788 - accuracy: 0.8655 - val_loss: 0.4499 - val_accuracy: 0.8480
Epoch 15/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3578 - accuracy: 0.8766 - val_loss: 0.4207 - val_accuracy: 0.8444
Epoch 16/40
248/248 [==============================] - 76s 306ms/step - loss: 0.3483 - accuracy: 0.8749 - val_loss: 0.5112 - val_accuracy: 0.8298
Epoch 17/40
248/248 [==============================] - 77s 310ms/step - loss: 0.3572 - accuracy: 0.8744 - val_loss: 0.4907 - val_accuracy: 0.8313
Epoch 18/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3533 - accuracy: 0.8777 - val_loss: 0.4622 - val_accuracy: 0.8348
Epoch 19/40
248/248 [==============================] - 78s 312ms/step - loss: 0.3546 - accuracy: 0.8759 - val_loss: 0.4387 - val_accuracy: 0.8475
Epoch 20/40
248/248 [==============================] - 80s 323ms/step - loss: 0.3412 - accuracy: 0.8756 - val_loss: 0.4191 - val_accuracy: 0.8455
Epoch 21/40
248/248 [==============================] - 78s 315ms/step - loss: 0.3386 - accuracy: 0.8793 - val_loss: 0.4500 - val_accuracy: 0.8429
Epoch 22/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3221 - accuracy: 0.8893 - val_loss: 0.4427 - val_accuracy: 0.8490
Epoch 23/40
248/248 [==============================] - 76s 306ms/step - loss: 0.3431 - accuracy: 0.8814 - val_loss: 0.4141 - val_accuracy: 0.8566
Epoch 24/40
248/248 [==============================] - 81s 325ms/step - loss: 0.3269 - accuracy: 0.8857 - val_loss: 0.4758 - val_accuracy: 0.8505
Epoch 25/40
248/248 [==============================] - 80s 323ms/step - loss: 0.3320 - accuracy: 0.8830 - val_loss: 0.4240 - val_accuracy: 0.8545
Epoch 26/40
248/248 [==============================] - 77s 311ms/step - loss: 0.3164 - accuracy: 0.8891 - val_loss: 0.4258 - val_accuracy: 0.8540
Epoch 27/40
248/248 [==============================] - 80s 322ms/step - loss: 0.3174 - accuracy: 0.8919 - val_loss: 0.4470 - val_accuracy: 0.8500
Epoch 28/40
248/248 [==============================] - 77s 310ms/step - loss: 0.3044 - accuracy: 0.8929 - val_loss: 0.4503 - val_accuracy: 0.8535
Epoch 29/40
248/248 [==============================] - 77s 309ms/step - loss: 0.3194 - accuracy: 0.8883 - val_loss: 0.4477 - val_accuracy: 0.8545
Epoch 30/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3119 - accuracy: 0.8909 - val_loss: 0.4205 - val_accuracy: 0.8495
Epoch 31/40
248/248 [==============================] - 77s 309ms/step - loss: 0.2947 - accuracy: 0.8961 - val_loss: 0.4215 - val_accuracy: 0.8566
Epoch 32/40
248/248 [==============================] - 77s 310ms/step - loss: 0.2862 - accuracy: 0.8987 - val_loss: 0.4422 - val_accuracy: 0.8455
Epoch 33/40
248/248 [==============================] - 77s 309ms/step - loss: 0.3128 - accuracy: 0.8934 - val_loss: 0.4183 - val_accuracy: 0.8581
Epoch 34/40
248/248 [==============================] - 76s 308ms/step - loss: 0.2953 - accuracy: 0.8952 - val_loss: 0.4067 - val_accuracy: 0.8667
Epoch 35/40
248/248 [==============================] - 77s 310ms/step - loss: 0.2937 - accuracy: 0.8981 - val_loss: 0.4417 - val_accuracy: 0.8525
Epoch 36/40
248/248 [==============================] - 78s 313ms/step - loss: 0.2927 - accuracy: 0.8979 - val_loss: 0.4562 - val_accuracy: 0.8439
Epoch 37/40
248/248 [==============================] - 79s 317ms/step - loss: 0.2903 - accuracy: 0.9000 - val_loss: 0.4307 - val_accuracy: 0.8545
Epoch 38/40
248/248 [==============================] - 81s 326ms/step - loss: 0.2911 - accuracy: 0.8966 - val_loss: 0.4528 - val_accuracy: 0.8530
Epoch 39/40
248/248 [==============================] - 78s 313ms/step - loss: 0.2829 - accuracy: 0.9024 - val_loss: 0.4411 - val_accuracy: 0.8515
Epoch 40/40
248/248 [==============================] - 77s 309ms/step - loss: 0.2863 - accuracy: 0.8958 - val_loss: 0.5121 - val_accuracy: 0.8399
Epoch 1/20
248/248 [==============================] - 77s 311ms/step - loss: 0.2772 - accuracy: 0.9028 - val_loss: 0.4174 - val_accuracy: 0.8596
Epoch 2/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2745 - accuracy: 0.9078 - val_loss: 0.4680 - val_accuracy: 0.8343
Epoch 3/20
248/248 [==============================] - 77s 311ms/step - loss: 0.2734 - accuracy: 0.9042 - val_loss: 0.4889 - val_accuracy: 0.8444
Epoch 4/20
248/248 [==============================] - 77s 308ms/step - loss: 0.2788 - accuracy: 0.9057 - val_loss: 0.4051 - val_accuracy: 0.8626
Epoch 5/20
248/248 [==============================] - 77s 310ms/step - loss: 0.2772 - accuracy: 0.9056 - val_loss: 0.4664 - val_accuracy: 0.8530
Epoch 6/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2738 - accuracy: 0.9053 - val_loss: 0.5409 - val_accuracy: 0.8258
Epoch 7/20
248/248 [==============================] - 77s 311ms/step - loss: 0.2612 - accuracy: 0.9082 - val_loss: 0.4639 - val_accuracy: 0.8591
Epoch 8/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2690 - accuracy: 0.9038 - val_loss: 0.4245 - val_accuracy: 0.8662
Epoch 9/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2611 - accuracy: 0.9087 - val_loss: 0.5023 - val_accuracy: 0.8419
Epoch 10/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2614 - accuracy: 0.9116 - val_loss: 0.4963 - val_accuracy: 0.8384
Epoch 11/20
248/248 [==============================] - 77s 310ms/step - loss: 0.2718 - accuracy: 0.9043 - val_loss: 0.4471 - val_accuracy: 0.8505
Epoch 12/20
248/248 [==============================] - 77s 308ms/step - loss: 0.2613 - accuracy: 0.9095 - val_loss: 0.4336 - val_accuracy: 0.8561
Epoch 13/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2543 - accuracy: 0.9111 - val_loss: 0.4350 - val_accuracy: 0.8455
Epoch 14/20
248/248 [==============================] - 77s 308ms/step - loss: 0.2552 - accuracy: 0.9086 - val_loss: 0.4328 - val_accuracy: 0.8556
Epoch 15/20
248/248 [==============================] - 80s 321ms/step - loss: 0.2379 - accuracy: 0.9165 - val_loss: 0.4548 - val_accuracy: 0.8505
Epoch 16/20
248/248 [==============================] - 79s 319ms/step - loss: 0.2585 - accuracy: 0.9109 - val_loss: 0.4483 - val_accuracy: 0.8556
Epoch 17/20
248/248 [==============================] - 77s 310ms/step - loss: 0.2436 - accuracy: 0.9162 - val_loss: 0.4635 - val_accuracy: 0.8561
Epoch 18/20
248/248 [==============================] - 77s 308ms/step - loss: 0.2471 - accuracy: 0.9155 - val_loss: 0.4757 - val_accuracy: 0.8485
Epoch 19/20
248/248 [==============================] - 77s 310ms/step - loss: 0.2563 - accuracy: 0.9129 - val_loss: 0.4316 - val_accuracy: 0.8657
Epoch 20/20
248/248 [==============================] - 79s 317ms/step - loss: 0.2434 - accuracy: 0.9139 - val_loss: 0.4313 - val_accuracy: 0.8566
4/4 [==============================] - 4s 33ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
accuracy = vit_classifier.evaluate(test_generator)
# Make predictions on test data
predictions = vit_classifier.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Get only the filenames without the directory path
test_generator.filenames = [os.path.basename(file) for file in test_generator.filenames]

# Add 1 to each predicted class to shift the numbering
predicted_classes_adjusted = predicted_classes + 1
# Create a DataFrame to store filenames without paths and predicted classes
results = pd.DataFrame({"image_id": test_generator.filenames, "label": predicted_classes_adjusted})

# Save results to a CSV file
results.to_csv('submission_rvit.csv', index=False)

# Display predicted classes
print(predicted_classes)

4/4 [==============================] - 5s 168ms/step
[2 2 4 0 2 0 4 2 1 3 1 4 0 4 0 0 0 4 3 3 2 3 2 0 4 0 2 4 4 1 0 2 3 0 4 4 0
 3 0 2 0 2 0 3 4 3 2 2 2 0 4 1 3 1 1 1 1 1 0 1 1 0 1 1 3 0 1 2 1 4 3 1 1 1
 1 1 2 1 1 0 3 2 3 0 3 2 3 4 4 2 3 2 4 3 3 4 3 3 2 0]


In [10]:
######ALEX NET

import keras

model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(5,activation='softmax')  
    
    
])

In [13]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']    
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 54, 54, 128)       46592     
                                                                 
 batch_normalization_5 (Batc  (None, 54, 54, 128)      512       
 hNormalization)                                                 
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 27, 27, 128)      0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 27, 27, 256)       819456    
                                                                 
 batch_normalization_6 (Batc  (None, 27, 27, 256)      1024      
 hNormalization)                                                 
                                                      

In [14]:
history=model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    
)

Epoch 1/50
248/248 [==============================] - 68s 271ms/step - loss: 1.8903 - accuracy: 0.4037 - val_loss: 1.6548 - val_accuracy: 0.2753
Epoch 2/50
248/248 [==============================] - 66s 266ms/step - loss: 1.2663 - accuracy: 0.4879 - val_loss: 2.0899 - val_accuracy: 0.2702
Epoch 3/50
248/248 [==============================] - 68s 272ms/step - loss: 1.1476 - accuracy: 0.5307 - val_loss: 1.4332 - val_accuracy: 0.3747
Epoch 4/50
248/248 [==============================] - 67s 269ms/step - loss: 1.1153 - accuracy: 0.5525 - val_loss: 1.2825 - val_accuracy: 0.4364
Epoch 5/50
248/248 [==============================] - 69s 277ms/step - loss: 1.0653 - accuracy: 0.5794 - val_loss: 1.2759 - val_accuracy: 0.4601
Epoch 6/50
248/248 [==============================] - 70s 282ms/step - loss: 1.0196 - accuracy: 0.6003 - val_loss: 1.3848 - val_accuracy: 0.4520
Epoch 7/50
248/248 [==============================] - 76s 307ms/step - loss: 1.0055 - accuracy: 0.6130 - val_loss: 1.1646 - val_ac